In [25]:
import requests
import json
from scrapy.http import HtmlResponse
import os
import datetime
import boto3
import copy
import hashlib

url = f"https://rewardsforjustice.net/index/?jsf=jet-engine:rewards-grid&tax=cyber:857&pagenum=1&nocache=1650868733"
# url = f"https://rewardsforjustice.net/index/?jsf=jet-engine%3Arewards-grid&tax=crime-category%3A1073&pagenum=1&nocache=1650868733"

payload = "action=jet_engine_ajax&handler=get_listing&page_settings%5Bpost_id%5D=22076&page_settings%5Bqueried_id%5D=22076%7CWP_Post&page_settings%5Belement_id%5D=ddd7ae9&page_settings%5Bpage%5D=1&listing_type=elementor&isEditMode=false&addedPostCSS%5B%5D=22078"
headers = {
    'authority': 'rewardsforjustice.net',
    'method': 'POST',
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-IN,en-GB;q=0.9,en-US;q=0.8,en;q=0.7,hi;q=0.6,gu;q=0.5',
    'content-type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36',
    'x-requested-with': 'XMLHttpRequest'
}

res = requests.post(url, headers=headers, data=payload)
body = json.loads(res.text)
data = body['data']['html']

In [26]:
ress = HtmlResponse(url="example.com",body=data,encoding='utf-8')
                
body = ress.xpath('//*[@class="jet-engine-listing-overlay-wrap"]//@data-url').extract()
body

['https://rewardsforjustice.net/rewards/foreign-malicious-cyber-activity-against-u-s-critical-infrastructure/',
 'https://rewardsforjustice.net/rewards/marat-valeryevich-tyukov/',
 'https://rewardsforjustice.net/rewards/mikhail-mikhailovich-gavrilov/',
 'https://rewardsforjustice.net/rewards/pavel-aleksandrovich-akulov/',
 'https://rewardsforjustice.net/rewards/evgeny-viktorovich-gladkikh/']

In [1]:
import json
import requests
from scrapy.http import HtmlResponse
import hashlib


url = "https://moneysmart.gov.au/companies-you-should-not-deal-with"

r = requests.get(url)
# resp = HtmlResponse(url="example.com",body=r.text, encoding='utf-8')

# for i in resp.xpath("https://moneysmart.gov.au/companies-you-should-not-deal-with"):
#     print(i.xpath(".//text()").get())


In [14]:
import json
import requests
from scrapy.http import HtmlResponse
import hashlib
from bs4 import BeautifulSoup

url = "https://moneysmart.gov.au/companies-you-should-not-deal-with"

r = requests.get(url)
resp = HtmlResponse(url="example.com",body=r.text, encoding='utf-8')

for i in resp.xpath("//div[@class='accordianRowContent']/div/h4/a"):
    print(i)


In [15]:
soup = BeautifulSoup(r.content,"lxml")